# Initializations
- Dataset columns should include "ProteinID" and "ModifiedLocationNum" 

In [1]:
from protds.protds_alphafold import *

alphafold_download_folder("/home/sduong/Desktop/alphafold-yeast") #specify where Alphafold downloads are

filename = 'En47C_Chaperone_allSites.csv'
path = r'data' #can replace "data" with any path 
data = pd.read_csv(os.path.join(path ,filename))
df = data[pd.notna(data.ModifiedLocationNum)].copy() #keep only rows with ModifiedLocationNumber

# Distances
- unit measure is 1 Angstrom
- for each Feature site, distance is defined by the euclidean distance between centerOfMass(ModifiedLocation) and centerOfMass(siteLocation)

In [2]:
#Calculate Distances (takes a while)
df["Feature"] = [[] for i in df.index]
df["Feature_Location"] = df["Feature"] = [[] for i in df.index]
df["Distance_to_ModifiedLocationNum"] = [[] for i in df.index]
dist_list = []; feat_list = []; feat_loc_list=[]

for i in df.index:
    entry = df.loc[i]
    feats, feat_locs, dists = get_distance_to_features(entry.ProteinID, entry.ModifiedLocationNum)
    dist_list.append(dists)
    feat_list.append(feats)
    feat_loc_list.append(feat_locs)
    
df["Feature"] = feat_list
df["Feature_Location"] = feat_loc_list
df["Distance_to_ModifiedLocationNum"] = dist_list

df_long = df.explode(column=["Feature", "Feature_Location", "Distance_to_ModifiedLocationNum"])

# Output
- summary and long tables saved in the output folder

In [3]:
#Save Results
output_path = r'output' #location of output directory
df.to_csv(os.path.join(output_path, ''.join(filename.split('.')[:-1])+"-summary.csv"), index=False)
df_long.to_csv(os.path.join(output_path, ''.join(filename.split('.')[:-1])+"-long.csv"), index=False)